In [ ]:
from newspaper import Article
from bs4 import BeautifulSoup
import requests
import urllib

#Hindu Business Line RSS Feed
url = "https://economictimes.indiatimes.com/rssfeedstopstories.cms"

resp = requests.get(url)

bsObj = BeautifulSoup(resp.content, features = "xml")
links = bsObj.findAll('link', text=True)

##Links extracted from RSS Feed
print('EXTRACTED LINKS FROM RSS FEED')
for link in links:
    if(link.get_text()=="https://economictimes.indiatimes.com"):
        links.remove(link)

for link in links:
    print(link.getText())


In [ ]:
#Using pagerank and textrank to get the summary

from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [ ]:
from sqlalchemy import *
db = create_engine('sqlite:///economicDB.db', echo=True)

metadata = MetaData(db)

news = Table('news', metadata,
            Column('id', Integer, primary_key=True),
            Column('paperName', String(100)),
            Column('title', String(250)),
            Column('summary', String(10000))
            )
news.create()

In [ ]:
import numpy as np
from nltk.tokenize import sent_tokenize
from operator import itemgetter 
import warnings
#warnings.filterwarnings("ignore")

#Store all text in list
article_text = []
article_title = []


for link in links:
    print(link)
    article = Article(link.getText(), language="en")
    article.download()
    article.parse()
    if(len(article.text.split('.'))<=3):
        continue
    article_text.append(article.text)
    article_title.append(article.title)
    print('added')
article_text.pop(0)
stop_words = stopwords.words('english')

def build_similarity_matrix(sentences, stopwords=None):
    
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    
    for idx in range(len(S)):
        if(S[idx].sum()==0):
            break
        S[idx] /= S[idx].sum()
 
    return S

def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P


temp = ""
counter = 0
i = news.insert()
for texts in article_text:
    sentences = sent_tokenize(texts)
    S = build_similarity_matrix(sentences, stop_words)  
    sentence_ranks = pagerank(S)
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    SUMMARY_SIZE = 3
    SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
    summary = itemgetter(*SELECTED_SENTENCES)(sentences)
    # Print the actual summary
    print('\nSUMMARY')
    print('------------------------------')
    for sentence in summary:
        temp+=(''.join(sentence))
        print(''.join(sentence), end="")
    print()
    i.execute({'paperName':'Economic times', 'title':article_title[counter],'summary':temp})
    counter+=1

In [ ]:
s = news.select()
rs = s.execute()

for row in rs:
    print(row.title, '\n\n', row.summary)
    print()
    print('====================================================================')